In [ ]:
%matplotlib widget

from collections import deque
from copy import deepcopy
from pathlib import Path

import flammkuchen as fl
import numpy as np
import pandas as pd
from bg_atlasapi import BrainGlobeAtlas
from bg_atlasapi.core import Atlas
from matplotlib import pyplot as plt
from motions.em import NeuroSkeleton, load_skeletons_from_zip
from motions.em.skeleton_mesh import make_full_neuron
from scipy.ndimage import gaussian_filter
from tqdm import tqdm

atlas = Atlas("/Users/luigipetrucco/.brainglobe/ipn_zfish_0.5um_v1.4")

from numba import jit, prange


@jit(nopython=True, parallel=True)
def _set_voxels(stack, idxs):
    for i in prange(idxs.shape[0]):
        stack[idxs[i, 0], idxs[i, 1], idxs[i, 2]] += 1

import lotr.plotting as pltltr

# Neuron plotter
from motions.em.plotting import AtlasPlotter

plotter = AtlasPlotter(
    structures=["lat_l_col", "ipn", "lat_r_col", "med_l_col", "med_r_col"]
)
# plotter = AtlasPlotter(structures=["ipn"])
plotter.shape_color = None
plotter.shape_fill = False
plotter.shape_linewidth = 0.2
plotter.shape_linecolor = (0.1,) * 3
plotter.shape_alpha = 1
plotter.font_size = 10

In [ ]:
master_folder = Path(r"J:\_Shared\experiments\E0060_ipn_em_tracing")
# master_folder =  Path(r"C:\Users\lpetrucco\Desktop")
master_folder = Path("/Users/luigipetrucco/Google Drive/data/ipn_tracing/")
data_folder = master_folder / "main_data_dir"
neurons_dict = fl.load(data_folder / "all_skeletons.h5")

working_folder = master_folder / "rosette"
rosette_volume = fl.load(working_folder / "volume.h5")

# Cell plots

In [ ]:
pairs = [
    ["n053", "n084"],
    ["p041", "p048"],
    ["p056", "p043"],
    ["p052", "p042"],
    ["n083", "n086"],
    ["n006", "n039"],
    ["n021", "n005"],
    ["n049", "n064"],
    ["n057", "n043"],
]
labels = []

In [ ]:
for _, sk in neurons_dict.items():
    if "four" in sk.cell_comments:
        print(sk.cell_comments)

In [ ]:
["n017", "n006", "n039"]

In [ ]:
cols = cols = np.array([[209, 116, 146], [0, 183, 163]]) / 255

In [ ]:
pair_combinations

In [ ]:
pair_combinations = [[pairs[0],] + [pairs[3],], pairs[4:-2]]

for n, pair_combo in enumerate(pair_combinations[:2]):
    f, cols_axs = plt.subplots(len(pair_combo), 3, figsize=(6, 2.0 * len(pair_combo)))

    for i, (axs, pair) in enumerate(zip(cols_axs, pair_combo)):
        plotter.generate_projection_plots(
            axs, labels=(i == len(pair_combo) - 1), edge=120, title=i == 0
        )
        print(pair)
        for cid, col in zip(pair, cols):
            plotter.plot_neurons(
                axs,
                neurons_dict[cid],
                color=col,
                linewidth=0.2,
                select="axon",
                soma_s=0,
                alpha=0.7,
                rasterized=True,
            )
            plotter.plot_neurons(
                axs,
                neurons_dict[cid],
                color=col,
                linewidth=0.5,
                select="dendrites",
                alpha=1,
                rasterized=True,
            )

    #f.savefig(
    #    f"/Users/luigipetrucco/Desktop/labmeetingfigs/ipn_neurons_{n}.pdf", dpi=300
    #)

In [ ]:
pair_combinations = [pairs[1:3]]

for pair_combo in pair_combinations:
    f, cols_axs = plt.subplots(len(pair_combo), 3, figsize=(8, 2.5 * len(pair_combo)))

    for i, (axs, pair) in enumerate(zip(cols_axs, pair_combo)):
        plotter.generate_projection_plots(
            axs, labels=(i == len(pair_combo) - 1), edge=175, title=i == 0
        )

        for cid, col in zip(pair, cols):
            plotter.plot_neurons(
                axs,
                neurons_dict[cid],
                color=col,
                linewidth=0.2,
                select="axon",
                soma_s=0,
                alpha=0.7,
            )
            plotter.plot_neurons(
                axs,
                neurons_dict[cid],
                color=col,
                linewidth=0.5,
                select="dendrites",
                alpha=1,
            )


f.savefig("/Users/luigipetrucco/Desktop/labmeetingfigs/aHB_neurons.pdf")

# Diffuse projection

In [ ]:
pairs = [
    ["n053", "n084"],  # gad1b
    ["p041", "p048"],
    ["p056", "p043"],
    ["p052", "p042"],
    ["n083", "n086"],
    ["n006", "n039"],
    ["n021", "n005"],
    ["n049", "n064"],  # posterior
    ["n057", "n043"],
]  # hemifield

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(8, 3))
plotter.generate_projection_plots(axs, labels=True, edge=175)

for cid, col in zip(["n049", "n064"], cols):  # zip(["n056", "n078"], cols):
    plotter.plot_neurons(
        axs,
        neurons_dict[cid],
        color=col,
        linewidth=0.2,
        select="axon",
        soma_s=0,
        alpha=0.7,
    )
    plotter.plot_neurons(
        axs, neurons_dict[cid], color=col, linewidth=0.5, select="dendrites", alpha=1
    )


f.savefig("/Users/luigipetrucco/Desktop/test.pdf")